In [1]:
from pytorch_lightning import Trainer
import active_catinous.FastGramDynamicMemoryLungNodule as braincatsmodel
from active_catinous.FastGramDynamicMemoryLungNodule import FastGramDynamicMemoryLungNodule
from datasets.LUNADataset import LUNADataset
from datasets.BrainAgeContinuous import BrainAgeContinuous

import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning as pl
import sklearn 
from sklearn.metrics import confusion_matrix, auc, roc_curve
import torch
import pandas as pd
import pickle
from py_jotools import mut, slurm, cache
import numpy as np
import gc

import hashlib
import dill
import active_catinous.utils as cutils

from sklearn.metrics import mean_absolute_error

from models.RetinaNetDetection import *

%load_ext autoreload
%autoreload 

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'datasets.LUNADatasetContinuous'

In [2]:
import pandas as pd

In [6]:
pd.read_csv('/project/catinous/lunadata/luna_lunacombined_dataset.csv').groupby(['res', 'split']).count()

Unnamed: 0  image  seriesuid  label  coordX  coordY  \
res split                                                             
geb test                78     78         78     78      78      78   
    train              312    312        312    312     312     312   
    val                 40     40         40     40      40      40   
ges base_train         498    498        498    498     498     498   
    test               208    208        208    208     208     208   
    train              244    244        244    244     244     244   
    val                104    104        104    104     104     104   
sie test                50     50         50     50      50      50   
    train              188    188        188    188     188     188   
    val                 22     22         22     22      22      22   

                diameter_mm  
res split                    
geb test                 78  
    train               312  
    val                  40  
ges base_train          498  
    test                208  
    train               244  
    val                 104  
sie test                 50  
    train               188  
    val                  22

In [7]:
pd.read_csv('/project/catinous/lunadata/luna_lunacombined_dataset.csv')

,Unnamed: 0,image,seriesuid,label,coordX,coordY,diameter_mm,res,split
0,0,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636...,1,316,367,23.350644,ges,base_train
1,1,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...,1,367,350,21.453729,ges,test
2,2,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...,1,308,198,8.785833,ges,test
3,3,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...,1,221,219,12.823120,ges,test
4,4,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.323541312620...,1,142,323,4.560710,ges,base_train
...,...,...,...,...,...,...,...,...,...
1739,1739,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.229096941293...,0,0,0,0.000000,sie,val
1740,1740,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.229096941293...,0,0,0,0.000000,sie,val
1741,1741,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.229096941293...,0,0,0,0.000000,sie,val
1742,1742,/data/dataset/lung/LIDC/DICOMS/LIDC-IDRI/LIDC-...,1.3.6.1.4.1.14519.5.2.1.6279.6001.675543413149...,0,0,0,0.000000,sie,train
